In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import tensorflow.compat.v1 as tf
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import sys
sys.path.append('..')
from detector.input_pipeline import PoseResidualNetworkPipeline

In [ ]:
cmap = plt.cm.get_cmap('autumn')
new_cmap = cmap(np.arange(cmap.N))
new_cmap[:, -1] = np.sqrt(np.linspace(0, 1, cmap.N))  # set alpha
cmap = ListedColormap(new_cmap)  # create new colormap

# Build a graph

In [ ]:
tf.reset_default_graph()

files = [
    '/home/dan/datasets/COCO/multiposenet/train/shard-0001.tfrecords',
    '/home/dan/datasets/COCO/multiposenet/train/shard-0002.tfrecords',
    '/home/dan/datasets/COCO/multiposenet/train/shard-0003.tfrecords',
]

with tf.device('/cpu:0'):
    pipeline = PoseResidualNetworkPipeline(files, is_training=True, batch_size=10, max_keypoints=None)
    dataset = pipeline.dataset
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
    init = iterator.make_initializer(dataset)
    features, labels = iterator.get_next()

print(features, labels)

# Show an image

In [ ]:
ORDER = {
    0: 'nose',
    1: 'left eye', 2: 'right eye',
    3: 'left ear', 4: 'right ear',
    5: 'left shoulder', 6: 'right shoulder',
    7: 'left elbow', 8: 'right elbow',
    9: 'left wrist', 10: 'right wrist',
    11: 'left hip', 12: 'right hip',
    13: 'left knee', 14: 'right knee',
    15: 'left ankle', 16: 'right ankle'
}


EDGES = [
    (0, 1), (0, 2),
    (1, 3), (2, 4),
    (5, 7), (7, 9), (6, 8), (8, 10),
    (11, 13), (13, 15), (12, 14), (14, 16),
    (3, 5), (4, 6),
    (5, 11), (6, 12)
]


def get_keypoints(heatmaps, box, threshold):
    """
    Arguments:
        heatmaps: a numpy float array with shape [h, w, 17].
        box: a numpy array with shape [4].
        threshold: a float number.
    Returns:
        a numpy int array with shape [17, 3].
    """
    keypoints = np.zeros([17, 3], dtype='int32')

    ymin, xmin, ymax, xmax = box
    height, width = ymax - ymin, xmax - xmin
    h, w, _ = heatmaps.shape

    for j in range(17):
        mask = heatmaps[:, :, j]
        if mask.max() > threshold:
            y, x = np.unravel_index(mask.argmax(), mask.shape)
            y = np.clip(int(y * height/h), 0, height)
            x = np.clip(int(x * width/w), 0, width)
            keypoints[j] = np.array([x, y, 1])

    return keypoints


def draw_pose(draw, keypoints, box):
    """
    Arguments:
        draw: an instance of ImageDraw.Draw.
        keypoints: a numpy int array with shape [17, 3].
        box: a numpy int array with shape [4].
    """
    ymin, xmin, ymax, xmax = box
    keypoints = keypoints.copy()
    keypoints += np.array([xmin, ymin, 0])

    for (p, q) in EDGES:

        x1, y1, v1 = keypoints[p]
        x2, y2, v2 = keypoints[q]

        both_visible = v1 > 0 and v2 > 0
        if both_visible:
            draw.line([(x1, y1), (x2, y2)], fill='red')

    for j in range(17):
        x, y, v = keypoints[j]
        if v > 0:
            s = 1
            draw.ellipse([
                (x - s, y - s),
                (x + s, y + s)
            ], fill='red')

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    output = sess.run([features, labels])

In [ ]:
i = 0
w, h = 36, 56
w, h = w * 5, h * 5

background = Image.new('RGBA', (w * 2, h * 17), (255, 255, 255, 255))
draw = ImageDraw.Draw(background, 'RGBA')

keypoints = get_keypoints(output[1][i], box=(0, 0, h, w), threshold=0.9)
heatmaps = (cmap(output[0][i]) * 255).astype('uint8')
binary_masks = (output[1][i] * 255).astype('uint8')

for j, name in ORDER.items():

    heat = Image.fromarray(heatmaps[:, :, j]).resize((w, h))
    mask = Image.fromarray(binary_masks[:, :, j]).resize((w, h))

    background.paste(heat, (0, j*h))
    background.paste(mask, (w, j*h))
    draw.text((0, j*h), name, fill='red')
    draw_pose(draw, keypoints, box=(j*h, w, j*h + h, 2 * w))
    
background